Extract data from weather web using web scrapping

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.timeanddate.com/weather/usa/new-york/historic"
response = requests.get(URL)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all table rows
rows = soup.find_all('tr')

# Find the table containing the weather data (adjust selector based on actual table structure)
table = soup.find('table')

# Check if the table is found
if table:
    # Extract table headers (column names)
    headers = [header.text.strip() for header in table.find_all('th')]

    # Print the headers for debugging
    print("Extracted Headers:", headers)
else:
    print("Table not found on the webpage.")


# List to store scraped data
web_scraped_data = []

# Loop through 
for row in rows[1:]: 
    columns = row.find_all('td')
    
    if len(columns) >= 7:  # Ensure there are enough columns
        try:
            # Extract and clean the data for each column
            temp = columns[1].text.strip()  
            weather = columns[2].text.strip()
            wind = columns[3].text.strip()
            humidity = columns[5].text.strip()
            barometer = columns[6].text.strip()
            visibility = columns[7].text.strip()

            # Append the data as a dictionary to the list
            web_scraped_data.append({
                'Temp': temp,
                'Weather': weather,
                'Wind': wind,
                'Humidity': humidity,
                'Barometer': barometer,
                'Visibility': visibility,
            })
        except Exception as e:
            print(f"Error processing row: {[col.text.strip() for col in columns]}, Error: {e}")

# Convert the list of dictionaries into a Pandas DataFrame
if web_scraped_data:
    web_scraping_df = pd.DataFrame(web_scraped_data)
    print(web_scraping_df)
else:
    print("No valid data was scraped.")




Extracted Headers: ['', 'Temperature', 'Humidity', 'Pressure', 'High', 'Low', 'Average']
     Temp            Weather     Wind Humidity  Barometer Visibility
0    1 °C             Clear.   7 km/h      40%  1019 mbar      16 km
1    1 °C             Clear.  13 km/h      38%  1019 mbar      16 km
2    1 °C             Clear.   6 km/h      38%  1019 mbar      16 km
3    1 °C             Clear.  13 km/h      35%  1019 mbar      16 km
4    1 °C             Clear.  13 km/h      35%  1018 mbar      16 km
5    2 °C             Clear.   7 km/h      34%  1018 mbar      16 km
6    2 °C    Passing clouds.  17 km/h      32%  1018 mbar      16 km
7    2 °C  Scattered clouds.      N/A      34%  1017 mbar      16 km
8    3 °C             Sunny.  20 km/h      31%  1017 mbar      16 km
9    3 °C             Sunny.  20 km/h      31%  1017 mbar      16 km
10   2 °C             Sunny.  13 km/h      37%  1017 mbar      16 km
11   1 °C             Sunny.      N/A      40%  1018 mbar      16 km
12   0 °C     

Store the data

In [8]:
import psycopg2
from psycopg2 import sql

# Database connection parameters
DB_PARAMS = {
    'dbname': 'weather',
    'user': 'postgres',
    'password': 'KARU55bime22',
    'host': 'localhost',  # Or your database host
    'port': '5432',       # Default PostgreSQL port
}

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**DB_PARAMS)
    cursor = conn.cursor()
    
    # SQL query to insert data into the weather_data table
    insert_query = sql.SQL("""
        INSERT INTO weather_details (temp, weather, wind, humidity, barometer, visibility)
        VALUES (%s, %s, %s, %s, %s, %s)
    """)
    
    # Insert each row of the DataFrame into the database
    for _, row in web_scraping_df.iterrows():
        cursor.execute(insert_query, (
            row['Temp'], 
            row['Weather'], 
            row['Wind'], 
            row['Humidity'], 
            row['Barometer'], 
            row['Visibility']
        ))
    
    # Commit the transaction
    conn.commit()
    print("Data successfully inserted into the database.")
    
except Exception as e:
    print(f"Error while inserting data: {e}")
    
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Data successfully inserted into the database.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the COVID-19 data webpage
URL = "https://www.worldometers.info/coronavirus/"

# Send a GET request to fetch the webpage content
response = requests.get(URL)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the COVID-19 data
table = soup.find('table', id='main_table_countries_today')

# Extract table headers (column names)
headers = [header.text.strip() for header in table.find_all('th')]

# Debugging: Print extracted headers
print("Extracted Headers:", headers)

# Define the columns of interest (match with extracted headers)
desired_columns = [
    "Country,Other",
    "TotalCases",
    "TotalDeaths",
    "NewCases",
    "NewDeaths",
    "TotalRecovered",
    "ActiveCases",
    "Deaths/1M pop",
    "Population"
]

# Find all rows in the table
rows = table.find('tbody').find_all('tr')

# List to store scraped data
covid_data = []

# Iterate through the rows and extract data
for row in rows:
    columns = row.find_all('td')
    if columns:
        data = [col.text.strip() for col in columns]
        if len(data) >= len(headers):
            covid_row = {headers[i]: data[i] for i in range(len(headers)) if headers[i] in desired_columns}
            covid_data.append(covid_row)

# Convert the list of dictionaries into a Pandas DataFrame
covid_df = pd.DataFrame(covid_data)

# Keep only the desired columns
available_columns = [col for col in desired_columns if col in covid_df.columns]
covid_df = covid_df[available_columns]

print(covid_df)


Extracted Headers: ['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl', 'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop']
      Country,Other   TotalCases TotalDeaths NewCases NewDeaths  \
0     North America  131,889,132   1,695,941                      
1              Asia  221,500,265   1,553,662                      
2            Europe  253,406,198   2,101,824                      
3     South America   70,200,879   1,367,332                      
4           Oceania   14,895,771      33,015                      
..              ...          ...         ...      ...       ...   
234         Tokelau           80                                  
235    Vatican City           29                                  
236  

Store the data

In [14]:
import psycopg2
from psycopg2 import sql

# PostgreSQL database connection details
DB_CONFIG = {
    "dbname": "covid",
    "user": "postgres",
    "password": "KARU55bime22",
    "host": "localhost",
    "port": "5432"
}

# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS covid_history (
        country TEXT,
        total_cases BIGINT,
        total_deaths BIGINT,
        new_cases BIGINT,
        new_deaths BIGINT,
        total_recovered BIGINT,
        active_cases BIGINT,
        deaths_per_million FLOAT,
        population BIGINT
    )
    """)

    # Insert data into the database
    for _, row in covid_df.iterrows():
        cursor.execute("""
        INSERT INTO covid_history (country, total_cases, total_deaths, new_cases, new_deaths, total_recovered, active_cases, deaths_per_million, population)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (country) DO NOTHING
        """, (
            row.get("Country,Other"),
            row.get("TotalCases"),
            row.get("TotalDeaths"),
            row.get("NewCases"),
            row.get("NewDeaths"),
            row.get("TotalRecovered"),
            row.get("ActiveCases"),
            row.get("Deaths/1M pop"),
            row.get("Population")
        ))

    # Commit changes
    conn.commit()

    print("Data inserted successfully!")

except Exception as e:
    print("Error while connecting to PostgreSQL:", e)

finally:
    if conn:
        cursor.close()
        conn.close()
        print("PostgreSQL connection closed.")


Data inserted successfully!
PostgreSQL connection closed.
